In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Conv2D, Dense, Dropout, Activation, Flatten, MaxPooling2D
from keras.optimizers import SGD

/home/fx/env/stock/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEM', 'BH', 'BJC', 'BTS', 'CENTEL', 'CPALL', 'CPF',
        'CPN', 'DTAC', 'EGCO', 'HMPRO', 'INTUCH', 
        'IRPC', 'KBANK', 'KCE', 'KKP', 'KTB', 'LH', 'MINT', 
        'PTT', 'PTTEP', 'ROBINS', 'SCB', 'SCC', 
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TRUE', 'TU']

In [3]:
lr = 0.1
epochs = 10000
batch_size = 100
optimizer = SGD(lr=lr)
activator = 'tanh'
loss_method = 'mse'
metrics = ['mae', 'accuracy']

In [4]:
def load_stock_data(symbol, year_start, year_end):
    stock_data = []
    with open('../Data set/SET50_OHLC/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        ma = []
        for row in file_data:
            if row[1] is '':
                continue
            elif len(ma) < 30:
                ma.append([float(row[1]), float(row[2]), float(row[3]), float(row[4])])
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                temp = [float(row[1]), float(row[2]), float(row[3]), float(row[4])]
                stock_data.append(find_diff_ma(ma, temp))
                del ma[0]
                ma.append(temp)
    return stock_data

def find_diff_ma(moving_averages, current_values):
    returns = []
    for i in range(len(moving_averages[0])):
        data = [moving_average[i] for moving_average in moving_averages]
        ma = np.sum(data) / 30
        diff_ma = current_values[i] - ma
        returns.append(diff_ma)
    return returns
        

In [5]:
stock_datas = [ load_stock_data(x, 2008, 2017) for x in SET50 ]

min_count = len(stock_datas[0])
rounds = min_count - 30
train_index = int(rounds * 0.7)

In [6]:
def create_model():
    model = Sequential()
    model.add(Conv2D(64, (7, 1), input_shape=(len(stock_datas), 30, 4), padding='same'))
    model.add(Activation(activator))
    model.add(MaxPooling2D(pool_size = (2, 1), data_format= "channels_last"))
    model.add(Flatten())
    model.add(Dense(1024))  
    model.add(Activation(activator))
    model.add(Dense(256))  
    model.add(Activation(activator))
    model.add(Dropout(0.2))
    model.add(Dense(len(stock_datas) * 4))
    
    return model

In [7]:
model = create_model()
model.summary()
model.compile(loss=loss_method, optimizer=optimizer, metrics=metrics)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 36, 30, 64)        1856      
_________________________________________________________________
activation_1 (Activation)    (None, 36, 30, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 30, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 34560)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              35390464  
_________________________________________________________________
activation_2 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               262400    
__________

In [8]:
data = []
count = 0
index = 0
while index < rounds:
    temp = [i[index : index + 30] for i in stock_datas]
    data.append(temp)
    index += 1
    
x = np.asarray(data)
x_train = x[0: train_index].astype('float32')
x_test = x[train_index: train_index + 1].astype('float32')

print(x_train.shape, x_test.shape)

(1690, 36, 30, 4) (1, 36, 30, 4)


In [9]:
target = []
index = 0
while index < rounds:
    target_data = [stock[index + 30] for stock in stock_datas]
    temp = []
    for data in target_data:
        temp = temp + data
    target.append(temp)
    index += 1
y = np.asarray(target)
y_train = y[0: train_index].astype('float32')
y_test = y[train_index: rounds].astype('float32')

print(y_train.shape, y_test.shape)

(1690, 144) (725, 144)


In [10]:
model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, verbose= 1)
model.save('CNNMAinput4output4.h5')

Epoch 1/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 22.8489 - mean_absolute_error: 2.3148 - acc: 0.0817
Epoch 2/10000
1690/1690 [==============================] - 0s 126us/step - loss: 17.2456 - mean_absolute_error: 2.0180 - acc: 0.1118
Epoch 3/10000
1690/1690 [==============================] - 0s 122us/step - loss: 14.1621 - mean_absolute_error: 1.8197 - acc: 0.1361
Epoch 4/10000
1690/1690 [==============================] - 0s 124us/step - loss: 15.2144 - mean_absolute_error: 1.8754 - acc: 0.1373
Epoch 5/10000
1690/1690 [==============================] - 0s 123us/step - loss: 13.0670 - mean_absolute_error: 1.7459 - acc: 0.1479
Epoch 6/10000
1690/1690 [==============================] - 0s 123us/step - loss: 11.0932 - mean_absolute_error: 1.5968 - acc: 0.1491
Epoch 7/10000
1690/1690 [==============================] - 0s 123us/step - loss: 9.9572 - mean_absolute_error: 1.5068 - acc: 0.1775
Epoch 8/10000
1690/1690 [==============================] - 0s 123us/step

1690/1690 [==============================] - 0s 124us/step - loss: 2.6527 - mean_absolute_error: 0.8765 - acc: 0.2615
Epoch 63/10000
1690/1690 [==============================] - 0s 124us/step - loss: 2.6116 - mean_absolute_error: 0.8684 - acc: 0.2598
Epoch 64/10000
1690/1690 [==============================] - 0s 123us/step - loss: 2.7381 - mean_absolute_error: 0.8796 - acc: 0.2663
Epoch 65/10000
1690/1690 [==============================] - 0s 124us/step - loss: 2.6057 - mean_absolute_error: 0.8673 - acc: 0.2675
Epoch 66/10000
1690/1690 [==============================] - 0s 124us/step - loss: 2.8216 - mean_absolute_error: 0.8869 - acc: 0.2586
Epoch 67/10000
1690/1690 [==============================] - 0s 123us/step - loss: 2.5668 - mean_absolute_error: 0.8610 - acc: 0.2692
Epoch 68/10000
1690/1690 [==============================] - 0s 123us/step - loss: 2.6213 - mean_absolute_error: 0.8658 - acc: 0.2675
Epoch 69/10000
1690/1690 [==============================] - 0s 123us/step - loss: 2.

1690/1690 [==============================] - 0s 124us/step - loss: 2.2048 - mean_absolute_error: 0.8068 - acc: 0.3260
Epoch 124/10000
1690/1690 [==============================] - 0s 123us/step - loss: 2.2108 - mean_absolute_error: 0.8039 - acc: 0.3083
Epoch 125/10000
1690/1690 [==============================] - 0s 123us/step - loss: 2.1349 - mean_absolute_error: 0.7989 - acc: 0.3272
Epoch 126/10000
1690/1690 [==============================] - 0s 124us/step - loss: 2.1636 - mean_absolute_error: 0.8006 - acc: 0.3219
Epoch 127/10000
1690/1690 [==============================] - 0s 123us/step - loss: 2.0985 - mean_absolute_error: 0.7938 - acc: 0.3154
Epoch 128/10000
1690/1690 [==============================] - 0s 124us/step - loss: 2.2175 - mean_absolute_error: 0.8041 - acc: 0.3260
Epoch 129/10000
1690/1690 [==============================] - 0s 124us/step - loss: 2.1155 - mean_absolute_error: 0.7959 - acc: 0.3373
Epoch 130/10000
1690/1690 [==============================] - 0s 124us/step - l

1690/1690 [==============================] - 0s 124us/step - loss: 1.9809 - mean_absolute_error: 0.7570 - acc: 0.3763
Epoch 185/10000
1690/1690 [==============================] - 0s 124us/step - loss: 2.0197 - mean_absolute_error: 0.7609 - acc: 0.3615
Epoch 186/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.9883 - mean_absolute_error: 0.7579 - acc: 0.3763
Epoch 187/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.9903 - mean_absolute_error: 0.7567 - acc: 0.3598
Epoch 188/10000
1690/1690 [==============================] - 0s 125us/step - loss: 1.9134 - mean_absolute_error: 0.7524 - acc: 0.3675
Epoch 189/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.9393 - mean_absolute_error: 0.7507 - acc: 0.3598
Epoch 190/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.9849 - mean_absolute_error: 0.7530 - acc: 0.3615
Epoch 191/10000
1690/1690 [==============================] - 0s 124us/step - l

1690/1690 [==============================] - 0s 124us/step - loss: 1.8342 - mean_absolute_error: 0.7303 - acc: 0.3959
Epoch 246/10000
1690/1690 [==============================] - 0s 123us/step - loss: 1.8742 - mean_absolute_error: 0.7330 - acc: 0.3840
Epoch 247/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.8926 - mean_absolute_error: 0.7342 - acc: 0.3858
Epoch 248/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.9159 - mean_absolute_error: 0.7365 - acc: 0.3870
Epoch 249/10000
1690/1690 [==============================] - 0s 123us/step - loss: 1.9273 - mean_absolute_error: 0.7388 - acc: 0.3852
Epoch 250/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.9585 - mean_absolute_error: 0.7373 - acc: 0.3716
Epoch 251/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.8521 - mean_absolute_error: 0.7278 - acc: 0.3651
Epoch 252/10000
1690/1690 [==============================] - 0s 125us/step - l

...

1690/1690 [==============================] - 0s 124us/step - loss: 1.2997 - mean_absolute_error: 0.6086 - acc: 0.5444
Epoch 9739/10000
1690/1690 [==============================] - 0s 125us/step - loss: 1.3576 - mean_absolute_error: 0.6170 - acc: 0.5438
Epoch 9740/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3457 - mean_absolute_error: 0.6143 - acc: 0.5544
Epoch 9741/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3283 - mean_absolute_error: 0.6112 - acc: 0.5669
Epoch 9742/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3766 - mean_absolute_error: 0.6142 - acc: 0.5456
Epoch 9743/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3316 - mean_absolute_error: 0.6148 - acc: 0.5550
Epoch 9744/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3359 - mean_absolute_error: 0.6148 - acc: 0.5515
Epoch 9745/10000
1690/1690 [==============================] - 0s 123us/s

1690/1690 [==============================] - 0s 124us/step - loss: 1.3487 - mean_absolute_error: 0.6121 - acc: 0.5615
Epoch 9799/10000
1690/1690 [==============================] - 0s 125us/step - loss: 1.3218 - mean_absolute_error: 0.6086 - acc: 0.5467
Epoch 9800/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3404 - mean_absolute_error: 0.6126 - acc: 0.5456
Epoch 9801/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3144 - mean_absolute_error: 0.6081 - acc: 0.5556
Epoch 9802/10000
1690/1690 [==============================] - 0s 123us/step - loss: 1.3132 - mean_absolute_error: 0.6099 - acc: 0.5556
Epoch 9803/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3007 - mean_absolute_error: 0.6094 - acc: 0.5379
Epoch 9804/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3433 - mean_absolute_error: 0.6157 - acc: 0.5396
Epoch 9805/10000
1690/1690 [==============================] - 0s 124us/s

1690/1690 [==============================] - 0s 124us/step - loss: 1.3418 - mean_absolute_error: 0.6140 - acc: 0.5598
Epoch 9859/10000
1690/1690 [==============================] - 0s 123us/step - loss: 1.3440 - mean_absolute_error: 0.6131 - acc: 0.5609
Epoch 9860/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3260 - mean_absolute_error: 0.6102 - acc: 0.5538
Epoch 9861/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3319 - mean_absolute_error: 0.6115 - acc: 0.5497
Epoch 9862/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3263 - mean_absolute_error: 0.6131 - acc: 0.5320
Epoch 9863/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3137 - mean_absolute_error: 0.6086 - acc: 0.5467
Epoch 9864/10000
1690/1690 [==============================] - 0s 125us/step - loss: 1.3208 - mean_absolute_error: 0.6127 - acc: 0.5680
Epoch 9865/10000
1690/1690 [==============================] - 0s 124us/s

1690/1690 [==============================] - 0s 124us/step - loss: 1.3116 - mean_absolute_error: 0.6123 - acc: 0.5385
Epoch 9919/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.4039 - mean_absolute_error: 0.6184 - acc: 0.5497
Epoch 9920/10000
1690/1690 [==============================] - 0s 125us/step - loss: 1.3086 - mean_absolute_error: 0.6093 - acc: 0.5621
Epoch 9921/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3206 - mean_absolute_error: 0.6091 - acc: 0.5544
Epoch 9922/10000
1690/1690 [==============================] - 0s 125us/step - loss: 1.3166 - mean_absolute_error: 0.6108 - acc: 0.5426
Epoch 9923/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3103 - mean_absolute_error: 0.6108 - acc: 0.5538
Epoch 9924/10000
1690/1690 [==============================] - 0s 125us/step - loss: 1.3401 - mean_absolute_error: 0.6182 - acc: 0.5402
Epoch 9925/10000
1690/1690 [==============================] - 0s 124us/s

1690/1690 [==============================] - 0s 124us/step - loss: 1.3732 - mean_absolute_error: 0.6126 - acc: 0.5521
Epoch 9979/10000
1690/1690 [==============================] - 0s 125us/step - loss: 1.3781 - mean_absolute_error: 0.6176 - acc: 0.5462
Epoch 9980/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3368 - mean_absolute_error: 0.6128 - acc: 0.5544
Epoch 9981/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3030 - mean_absolute_error: 0.6125 - acc: 0.5485
Epoch 9982/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3040 - mean_absolute_error: 0.6092 - acc: 0.5544
Epoch 9983/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3480 - mean_absolute_error: 0.6146 - acc: 0.5615
Epoch 9984/10000
1690/1690 [==============================] - 0s 124us/step - loss: 1.3336 - mean_absolute_error: 0.6106 - acc: 0.5562
Epoch 9985/10000
1690/1690 [==============================] - 0s 124us/s